In [1]:
# requires
# pip install git+https://github.com/Phylliida/MambaLens.git

from mamba_lens import HookedMamba # this will take a little while to import

model_path = "state-spaces/mamba-370m"
model = HookedMamba.from_pretrained(model_path, device='cuda')

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Moving model to device:  cuda


In [7]:
# remember to do
# pip install -e .
# in the root directory of this repo

from acdc.data.ioi import ioi_data_generator, ABC_TEMPLATES, get_all_single_name_abc_patching_formats
from acdc.data.utils import generate_dataset

num_patching_pairs = 500
seed = 27
valid_seed = 28
constrain_to_answers = True
has_symmetric_patching = True

templates = ABC_TEMPLATES
patching_formats = list(get_all_single_name_abc_patching_formats())

data = generate_dataset(model=model,
                  data_generator=ioi_data_generator,
                  num_patching_pairs=num_patching_pairs,
                  seed=seed,
                  valid_seed=valid_seed,
                  constrain_to_answers=constrain_to_answers,
                  has_symmetric_patching=has_symmetric_patching, 
                  varying_data_lengths=True,
                  templates=templates,
                  patching_formats=patching_formats)

In [ ]:

for i in range(10):
    # because there is padding if lengths vary, this only fetches the tokens that are part of the sequence
    toks = data.data[i][:data.last_token_position[i]+1]
    print(model.tokenizer.decode(toks))

from acdc import accuracy_metric
from acdc import ACDCEvalData
def logging_metric(data: ACDCEvalData):
    for data_subset in [data.patched, data.corrupted]:
        batch, _ = data_subset.data.size()
        for b in range(batch):
            toks = data_subset.data[b][:data_subset.last_token_position[b]+1]
            print(model.tokenizer.decode(toks))
            if data_subset.top_is_correct[b].item():
                print("correct prs:")
                for i, tok in enumerate(data_subset.correct[b]):
                    print(data_subset.correct_prs[b,i].item(), model.tokenizer.decode([tok.item()]))
                print("incorrect prs:")
                for i, tok in enumerate(data_subset.incorrect[b]):
                    print(data_subset.incorrect_prs[b,i].item(), model.tokenizer.decode([tok.item()]))
    return accuracy_metric(data=data)

data.eval(model=model, batch_size=10, metric=logging_metric)

<|endoftext|>Friends Lily, Donovan and Brian went to the house. Lily and Donovan gave a apple to
<|endoftext|>Friends Lily, Donovan and Diana went to the house. Lily and Donovan gave a apple to
<|endoftext|>When Owen, Catherine and Henry arrived at the office, Owen and Henry gave a apple to
<|endoftext|>When Owen, Jordan and Henry arrived at the office, Owen and Henry gave a apple to
<|endoftext|>When Miranda, Marco and Edgar arrived at the office, Marco and Edgar gave a kiss to
<|endoftext|>When Edwin, Marco and Edgar arrived at the office, Marco and Edgar gave a kiss to
<|endoftext|>Then, Mark, Matthew and Luke went to the station. Matthew and Luke gave a necklace to
<|endoftext|>Then, Edward, Matthew and Luke went to the station. Matthew and Luke gave a necklace to
<|endoftext|>When Thomas, Preston and Peter arrived at the store, Preston and Peter gave a snack to
<|endoftext|>When Shane, Preston and Peter arrived at the store, Preston and Peter gave a snack to
<|endoftext|>Friends L